In [ ]:
class etc_features(Dataset):
    
            
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)

        
    def create_features(self):
        self.data['item_firstday'] = self.data.groupby(['item_id'])['d'].transform('min')
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['item_firstday'])


#UMAPで商品波形をクラスタリングする目的で作成、採用せず
    def create_umap_features(self):
        self.tmp2 = pd.read_pickle('../data/sales_train_validation.pickle')
        self.data['store_umap1_cos'] = 0
        self.data['store_umap2_cos'] = 0
        for store in tmp2['store_id'].unique():
            tmp3 = tmp2[tmp2['store_id'] == store].iloc[:,6:]
            embedding = umap.UMAP(metric = 'cosine').fit_transform(tmp3)
            tmp2.iloc[tmp2[tmp2['store_id'] == store].index , 1919] = embedding[:,0]
            tmp2.iloc[tmp2[tmp2['store_id'] == store].index , 1920] = embedding[:,1]
        tmp2['id'] = tmp2['id'].apply(lambda x: x.replace('validation' , 'evaluation'))
        self.data = self.data.merge(right = tmp2[['id' , 'store_umap1_cos' , 'store_umap2_cos']] , on = 'id' , how = 'left' )
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['store_umap1_cos'])
        self.save(feature=self.data['store_umap2_cos'])


#売り上げ最大値からのラグ、最小値からのラグ
    def create_features(self):
        self.data['tmp'] = self.data.groupby(['id']).agg({'demand': ['cummax']})
        self.data['tmp_shift1'] = self.data.groupby(['id'])['tmp'].shift(1)
        demand_max = np.array(self.data['tmp'])
        demand_max_shift1 = np.array(self.data['tmp_shift1'])
        import math
        cnt_list = []
        for i in range(len(demand_max)):
            if math.isnan(demand_max_shift1[i]):
                cnt_list.append(0)
            else:
                if demand_max[i] == demand_max_shift1[i]:
                    cnt_list.append(0)
                else:
                    cnt_list.append(1)

        self.data['tmp2'] = cnt_list
        self.data['demand_max_change'] = self.data.groupby(['id']).agg({'tmp2': ['cumsum']})
        self.data['demand_max_lag'] = self.data.groupby(['demand_max_change' , 'id']).cumcount()
        self.data['demand_max_lag_shift1'] = self.data.groupby(['id'])['demand_max_lag'].shift(1)
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['demand_max_lag_shift1'])

    def create_min_features(self):
        self.data['tmp'] = self.data.groupby(['id']).agg({'demand': ['cummin']})
        self.data['tmp_shift1'] = self.data.groupby(['id'])['tmp'].shift(1)
        demand_max = np.array(self.data['tmp'])
        demand_max_shift1 = np.array(self.data['tmp_shift1'])
        import math
        cnt_list = []
        for i in range(len(demand_max)):
            if math.isnan(demand_max_shift1[i]):
                cnt_list.append(0)
            else:
                if demand_max[i] == demand_max_shift1[i]:
                    cnt_list.append(0)
                else:
                    cnt_list.append(1)

        self.data['tmp2'] = cnt_list
        self.data['demand_min_change'] = self.data.groupby(['id']).agg({'tmp2': ['cumsum']})
        self.data['demand_min_lag'] = self.data.groupby(['demand_min_change' , 'id']).cumcount()
        self.data['demand_min_lag_shift1'] = self.data.groupby(['id'])['demand_min_lag'].shift(1)
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['demand_min_lag_shift1'])
              
    def shift_features(self):
            with open('../features/etc_features' +'/'+ 'demand_min_lag_shift1' + '.joblib', mode="rb") as f:
                tmp = joblib.load(f)
                self.data = pd.concat([self.data,tmp],axis=1)
            for day in range(2,29):
                self.data['demand_min_lag_shift{0}'.format(day)] = self.data.groupby(['id'])['demand_min_lag_shift1'].shift(day-1)
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data['demand_min_lag_shift{0}'.format(day)])
                self.data = self.data.drop(columns = 'demand_min_lag_shift{0}'.format(day))


            with open('../features/etc_features' +'/'+ 'demand_max_lag_shift1' + '.joblib', mode="rb") as f:
                tmp = joblib.load(f)
                self.data = pd.concat([self.data,tmp],axis=1)
            for day in range(2,29):
                self.data['demand_max_lag_shift{0}'.format(day)] = self.data.groupby(['id'])['demand_max_lag_shift1'].shift(day-1)
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data['demand_max_lag_shift{0}'.format(day)])
                self.data = self.data.drop(columns = 'demand_max_lag_shift{0}'.format(day) )

In [ ]:
class rolling_lag_Feature(Dataset):
    #単純な移動平均

        
        
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)

    #移動平均
    def create_features(self):
        for i in range(1,28):
            DAYS_PRED = i+1
            for size in [7, 28, 56, 84, 168]:
                self.data[f"rolling_lag_mean_t{size}_shift{DAYS_PRED}"] = self.data.groupby(["id"]
                                                )["demand"].transform(lambda x: x.shift(DAYS_PRED).rolling(size).mean())
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data[f"rolling_lag_mean_t{size}_shift{DAYS_PRED}"])
      
            for size in [7, 28, 56, 84, 168]:
                self.data[f"rolling_lag_sum_t{size}_shift{DAYS_PRED}"] = self.data.groupby(["id"]
                                                )["demand"].transform(lambda x: x.shift(DAYS_PRED).rolling(size).sum())
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data[f"rolling_lag_sum_t{size}_shift{DAYS_PRED}"])
              
            for size in [7, 28, 56, 84, 168]:
                self.data[f"rolling_lag_std_t{size}_shift{DAYS_PRED}"] = self.data.groupby(["id"]
                                                )["demand"].transform(lambda x: x.shift(DAYS_PRED).rolling(size).std())
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data[f"rolling_lag_std_t{size}_shift{DAYS_PRED}"])
                
            for size in [7, 28, 56, 84, 168]:
                self.data[f"rolling_lag_max_t{size}_shift{DAYS_PRED}"] = self.data.groupby(["id"]
                                                )["demand"].transform(lambda x: x.shift(DAYS_PRED).rolling(size).max())
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data[f"rolling_lag_max_t{size}_shift{DAYS_PRED}"])

            for size in [7, 28, 56, 84, 168]:
                self.data[f"rolling_lag_min_t{size}_shift{DAYS_PRED}"] = self.data.groupby(["id"]
                                                )["demand"].transform(lambda x: x.shift(DAYS_PRED).rolling(size).min())
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data[f"rolling_lag_min_t{size}_shift{DAYS_PRED}"])

In [ ]:
class calendar_feature(Dataset):
    #
        
        
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__ 
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)

      #カレンダー情報、カーネルをそのまま持ってきた
    def create_feature(self):
        dt_col = "date"
        self.data[dt_col] = pd.to_datetime(self.data[dt_col])

        attrs = [
            "year",
            "quarter",
            "month",
            "week",
            "day",
            "dayofweek",
            "is_year_end",
            "is_year_start",
            "is_quarter_end",
            "is_quarter_start",
            "is_month_end",
            "is_month_start",]

        for attr in attrs:
            dtype = np.int16 if attr == "year" else np.int8
            self.data[attr] = getattr(self.data[dt_col].dt, attr).astype(dtype)
            
            self.data = self.reduce_mem_usage(self.data)
            self.save(feature=self.data[attr])
        

In [ ]:
class price_feature(Dataset):
    #
        
        
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__ 
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)
        
    #priceに関する基本的な移動平均特徴量
    def create_price_feature(self):
        
        DAYS_PRED = 28
        for size in [7, 28, 56, 84, 168]:
            self.data[f"rolling_price_mean_t{size}"] = self.data.groupby(["id"]
                                            )["sell_price"].transform(lambda x: x.rolling(size).mean())
            self.data = self.reduce_mem_usage(self.data)
            self.save(feature=self.data[f"rolling_price_mean_t{size}"])
            
        for size in [7, 28, 56, 84, 168]:
            self.data[f"rolling_price_std_t{size}"] = self.data.groupby(["id"]
                                            )["sell_price"].transform(lambda x: x.rolling(size).std())
            self.data = self.reduce_mem_usage(self.data)
            self.save(feature=self.data[f"rolling_price_std_t{size}"])
            
        for size in [7, 28, 56, 84, 168]:
            self.data[f"rolling_price_max_t{size}"] = self.data.groupby(["id"]
                                            )["sell_price"].transform(lambda x: x.rolling(size).max())
            self.data = self.reduce_mem_usage(self.data)
            self.save(feature=self.data[f"rolling_price_max_t{size}"])

        for size in [7, 28, 56, 84, 168]:
            self.data[f"rolling_price_min_t{size}"] = self.data.groupby(["id"]
                                            )["sell_price"].transform(lambda x: x.rolling(size).min())
            self.data = self.reduce_mem_usage(self.data)
            self.save(feature=self.data[f"rolling_price_min_t{size}"])
        
        
    #pirceの上下のカウント数と変化してからのラグ、かなり効いた効いた    
    def create_price_feature3(self):
        tmp = self.data.groupby(['id']).agg({'sell_price': ['cumsum', 'cumcount']})
        self.data['price_mean'] = (tmp[('sell_price', 'cumsum')] / (tmp[('sell_price', 'cumcount')] + 1))
        self.data['price_max'] = self.data.groupby(['id']).agg({'sell_price': ['cummax']})
        self.data['price_min'] = self.data.groupby(['id']).agg({'sell_price': ['cummin']})
        self.data['price_norm'] = self.data['sell_price'] / self.data['price_mean']
        
        
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['price_mean'])
        self.save(feature=self.data['price_max'])
        self.save(feature=self.data['price_min'])
        self.save(feature=self.data['price_norm'])

        self.data['sell_price_shift1'] = self.data.groupby(['id'])['sell_price'].shift(1)
        sell_price = np.array(self.data['sell_price'])
        sell_price_shift1 = np.array(self.data['sell_price_shift1'])
        import math
        cnt_list_up = []
        cnt_list_down = []
        for i in range(len(sell_price)):
            if math.isnan(sell_price_shift1[i]):
                cnt_list_up.append(0)
                cnt_list_down.append(0)
            else:
                if sell_price[i] > sell_price_shift1[i]:
                    cnt_list_up.append(1)
                    cnt_list_down.append(0)
                elif sell_price[i] < sell_price_shift1[i]:
                    cnt_list_up.append(0)
                    cnt_list_down.append(1)
                else:
                    cnt_list_up.append(0)
                    cnt_list_down.append(0)
        self.data['up'] = cnt_list_up
        self.data['down'] = cnt_list_down
        self.data['cnt_price_change_up'] = self.data.groupby(['id']).agg({'up': ['cumsum']})
        self.data['cnt_price_change_down'] = self.data.groupby(['id']).agg({'down': ['cumsum']})
        self.data['price_change_up_lag'] = self.data.groupby(['cnt_price_change_up' , 'id']).cumcount()
        self.data['price_change_down_lag'] = self.data.groupby(['cnt_price_change_down' , 'id']).cumcount()
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['cnt_price_change_up'])
        self.save(feature=self.data['price_change_up_lag'])
        self.save(feature=self.data['cnt_price_change_down'])
        self.save(feature=self.data['price_change_down_lag'])





In [ ]:
class demand_probability(Dataset):

        
        
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__ 
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)
        

##各売り上げ個数の出現確率、売り上げの確率分布の情報をモデルに教えるために作成。効果あり採用した
    def create_feature(self):
      for size in range(4,11):
          def tmp(x):
            if x == size:
              return 1
            else:
              return 0

          self.data['tmp'] = self.data['demand'].apply(lambda x: tmp(x))  
          self.data['tmp2'] = 1     
          self.data['length'] = self.data.groupby(['id']).agg({'tmp2': ['cumsum']})
          self.data['demand_{0}_prob'.format(size)] = self.data.groupby(['id']).agg({'tmp': ['cumsum']})
          self.data['demand_{0}_prob'.format(size)] /= self.data['length']
          self.data['demand_{0}_prob_shift1'.format(size)] = self.data.groupby(['id'])['demand_{0}_prob'.format(size)].shift(1)
          self.data = self.reduce_mem_usage(self.data)
          self.save(feature=self.data['demand_{0}_prob_shift1'.format(size)])


    def shift_features(self):
        for size in range(11):
            with open('../features/demand_probability' +'/'+ 'demand_{0}_prob_shift1'.format(size) + '.joblib', mode="rb") as f:
                tmp = joblib.load(f)
                self.data = pd.concat([self.data,tmp],axis=1)
            for day in range(2,29):
                self.data['demand_{0}_prob_shift{1}'.format(size , day)] = self.data.groupby(['id'])['demand_{0}_prob_shift1'.format(size)].shift(day-1)
                self.data = self.reduce_mem_usage(self.data)
                self.save(feature=self.data['demand_{0}_prob_shift{1}'.format(size , day)])
                self.data = self.data.drop(columns = 'demand_{0}_prob_shift{1}'.format(size , day))

In [ ]:
class Ordered_TS_mean_encoding(Dataset):
    #
        
        
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__ 
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)
        
        
#targetencoding、リークしないようにTSを採用。効果めちゃくちゃあり、特にprice,idでのencoding効果高かった
    def create_Ordered_TS_mean_encoding(self):
        self.data['demand'] = self.data['demand'].astype(np.int32)

        tmp = self.data.groupby(['id']).agg({'demand': ['cumsum', 'cumcount']})
        self.data['Ordered_TS_id'] = (tmp[('demand', 'cumsum')] / (tmp[('demand', 'cumcount')] + 1))
        self.data['Ordered_TS_id'] = self.data.groupby(['id'])['Ordered_TS_id'].shift(28)
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['Ordered_TS_id'])
        
        tmp = self.data.groupby(['id' , 'sell_price']).agg({'demand': ['cumsum', 'cumcount']})
        self.data['Ordered_TS_id_price'] = (tmp[('demand', 'cumsum')] / (tmp[('demand', 'cumcount')] + 1))
        for i in range(28):
            DAYS_PRED = i+1
            self.data['Ordered_TS_id_price_shift{0}'.format(DAYS_PRED)] = self.data.groupby(['id' , 'sell_price'])['Ordered_TS_id_price'].shift(DAYS_PRED)
            self.data = self.reduce_mem_usage(self.data)
            self.save(feature=self.data['Ordered_TS_id_price_shift{0}'.format(DAYS_PRED)])
            

In [ ]:
class SNAP_Feature(Dataset):
    #
        
        
    def __init__(self):
        super().__init__()
        self.file_dir = self.__class__.__name__ 
        if not os.path.exists(self.dir + self.file_dir):
            os.mkdir(self.dir + self.file_dir)
        
        
    
    def save(self ,feature):
        with open(self.dir + self.file_dir+'/'+'{0}.joblib'.format(feature.name), mode="wb") as f:
            joblib.dump(feature, f, compress=3)
        
    #SNAPからのラグ、特別な日からのラグはよく効きました。
    def create_features_SNAP_LAG(self):
        tmp = self.data.groupby(['date'])['snap_CA'].mean()
        lag_CA = []
        tmp2 = 0
        for i in tmp.values:
            if i == 0:
                tmp2 += 1
                lag_CA.append(tmp2)
            else:
                lag_CA.append(0)
                tmp2 = 0
                    
        tmp = self.data.groupby(['date'])['snap_TX'].mean()
        lag_TX = []
        tmp2 = 0
        for i in tmp.values:
            if i == 0:
                tmp2 += 1
                lag_TX.append(tmp2)
            else:
                lag_TX.append(0)
                tmp2 = 0
                
        tmp = self.data.groupby(['date'])['snap_WI'].mean()
        lag_WI = []
        tmp2 = 0
        for i in tmp.values:
            if i == 0:
                tmp2 += 1
                lag_WI.append(tmp2)
            else:
                lag_WI.append(0)
                tmp2 = 0
        self.data =  self.data['date']
        
        a= pd.DataFrame({'Lag_SNAP_CA':lag_CA,'Lag_SNAP_TX':lag_TX,'Lag_SNAP_WI':lag_WI,'date':tmp.index})
        self.data = pd.merge(self.data,a,how = 'left' ,on='date')
        self.data = self.reduce_mem_usage(self.data)
        self.save(feature=self.data['Lag_SNAP_CA'])
        self.save(feature=self.data['Lag_SNAP_TX'])
        self.save(feature=self.data['Lag_SNAP_WI'])